In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [121]:
tracklist = sorted(glob.glob(path+'tracks_nio/*2011*'))

In [122]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20111029.csv']

In [123]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2011.nc',datapath+'/rhum/rhum_2011.nc'])

In [124]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [126]:
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

####change year below##########################
dm2,tracklist,era5_date,era5_hour = readyear(2011) 

TCsl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCsl_ts.append(tsdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [127]:
##########radiation variables############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCrl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCrl_ts.append(tsdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [128]:
##########precip, mslp, sst variables############ we dont need this anymore.
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2011.nc',datapath+'/mslp/mslp_2011.nc',datapath+'/sst/sst_2011.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [129]:
#For vars with 21 vertical levels vorticity humidity file##################
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [130]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2011.nc')

In [131]:
#geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [132]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2011.nc')

In [133]:
#Vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [134]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [135]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2011.nc')

In [136]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [137]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20111029.csv']

In [138]:
#storms 3
maindf_keila=pd.read_csv(target+'2011_w10max_nio_keila.csv',delimiter=r",")
maindf_keila.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_keila=maindf_keila.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2011_nio_keila.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_keila=pd.DataFrame.from_dict(TCrl_ts[0])
prlev_keila=pd.DataFrame.from_dict(TCpr_ts[0])
slev_keila=pd.DataFrame.from_dict(TCsl_ts[0])
prlev_keila['mslp']=prlev_keila['mslp']/100

ts1_keila=pd.concat([maindf_keila,extra,slev_keila,rlev_keila,prlev_keila], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[0]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[0]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[0]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[0]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_keila=pd.DataFrame.from_dict(alldict)

ts_keila=pd.concat([ts1_keila,plev_keila], axis=1, join='inner')

ts_keila

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,12.055099,2.367005e-06,5.928226e-06,-6.653256e-06,-1.958938e-05,5.809564e-06,6.750427e-06,335.612885,350.371460,348.912720,...,-0.114145,-0.083229,-0.081476,-0.145472,-0.164502,-0.126555,-0.107114,-0.079677,-0.041862,-0.017086
1,11.664788,5.151447e-06,6.038431e-06,-5.926158e-06,-1.407261e-05,5.612488e-06,-2.388915e-06,334.140625,350.117493,348.646912,...,-0.200835,-0.199220,-0.166418,-0.208076,-0.183940,-0.150677,-0.126682,-0.103045,-0.061364,-0.028243
2,12.076045,4.491908e-06,-2.377669e-06,-8.104904e-06,-7.200085e-06,5.848245e-07,-2.430386e-06,336.036804,349.808380,348.299927,...,-0.162508,-0.135330,0.022709,0.003880,-0.036758,-0.057978,-0.060572,-0.055118,-0.023394,0.002154
3,11.888578,-8.037207e-07,3.972694e-06,-3.771493e-06,-6.648295e-06,9.458456e-06,4.493251e-06,334.918274,350.013275,348.165985,...,-0.111367,0.013724,0.028094,-0.058250,-0.138762,-0.138049,-0.083506,-0.056029,-0.010923,0.019067
4,12.550705,-4.033099e-06,1.094998e-05,-5.173708e-06,-1.056366e-05,9.619600e-06,4.110853e-06,331.787292,350.863007,348.548065,...,-0.094964,0.000930,0.015226,-0.052462,-0.098118,-0.055243,-0.060335,-0.056576,-0.031717,-0.012182
5,12.083922,-2.131600e-06,3.880810e-06,-8.327203e-06,-1.741038e-05,2.140549e-06,-1.327173e-06,331.667297,351.519653,348.702576,...,-0.026596,0.046803,0.049514,0.031141,0.007303,0.015905,-0.056174,-0.060719,-0.038657,-0.018434
6,12.904962,-2.631009e-06,-4.133747e-06,-1.010388e-05,-1.310438e-05,2.687535e-06,4.155208e-06,332.230042,351.523895,348.708069,...,0.006848,0.017619,0.029480,0.021928,-0.036693,-0.045923,-0.065072,-0.045269,-0.012987,0.009867
7,12.650217,1.991523e-05,-4.412064e-06,-1.392189e-05,-1.397591e-05,-1.765564e-06,4.098732e-06,334.121277,351.761749,348.712006,...,-0.078932,-0.044593,-0.116691,-0.216160,-0.274994,-0.240575,-0.139218,-0.082010,-0.024435,0.009276
8,13.250352,3.119516e-05,-4.678834e-06,-2.291874e-05,-9.080880e-06,-2.671243e-06,-2.992625e-06,335.638306,351.258179,348.473633,...,-0.162021,-0.125599,-0.137325,-0.190283,-0.208014,-0.207852,-0.149076,-0.109464,-0.052897,-0.016766
9,13.007581,2.157145e-05,-2.252393e-06,-1.759384e-05,-4.744627e-07,-2.884089e-06,-1.037025e-06,333.888184,350.835266,348.285980,...,-0.185950,-0.106297,-0.048027,-0.025947,-0.022787,-0.047449,-0.059472,-0.075904,-0.045156,-0.017241


In [139]:
ts_keila.to_csv(output+'inner/inner_nio_all_keila.csv')

In [115]:
maindf_phailin=pd.read_csv(target+'2013_w10max_nio_phailin.csv',delimiter=r",")
maindf_phailin.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_phailin=maindf_phailin.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2013_nio_phailin.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_phailin=pd.DataFrame.from_dict(TCrl_ts[1])
prlev_phailin=pd.DataFrame.from_dict(TCpr_ts[1])
slev_phailin=pd.DataFrame.from_dict(TCsl_ts[1])
prlev_phailin['mslp']=prlev_phailin['mslp']/100

ts1_phailin=pd.concat([maindf_phailin,extra,slev_phailin,rlev_phailin,prlev_phailin], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[1]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[1]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[1]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[1]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_phailin=pd.DataFrame.from_dict(alldict)

ts_phailin=pd.concat([ts1_phailin,plev_phailin], axis=1, join='inner')

ts_phailin

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,12.276583,0.000029,0.000048,1.633112e-05,2.857541e-07,-4.733704e-06,-2.174062e-06,333.454468,350.030762,348.756775,...,-0.466025,-0.495273,-0.420479,-0.401602,-0.337964,-0.274834,-0.229616,-0.163474,-0.050276,0.015990
1,12.332129,0.000036,0.000053,2.187599e-05,-8.693290e-07,-5.573118e-06,-3.450627e-06,333.575043,349.363861,348.265198,...,-0.514677,-0.558454,-0.495897,-0.470228,-0.416251,-0.380857,-0.301475,-0.209790,-0.089739,-0.012259
2,13.427410,0.000043,0.000052,1.999416e-05,6.500800e-06,-1.089153e-06,-6.368437e-07,333.535156,348.932495,347.935699,...,-0.513016,-0.581453,-0.581409,-0.599178,-0.565406,-0.527031,-0.368921,-0.234514,-0.101033,-0.030966
3,13.364658,0.000060,0.000057,2.090617e-05,1.322057e-05,-7.693538e-06,-3.845886e-06,333.355042,348.859222,348.001923,...,-0.640468,-0.726143,-0.727084,-0.669096,-0.632348,-0.561518,-0.368658,-0.226787,-0.092350,-0.019943
4,14.744117,0.000080,0.000047,1.813365e-05,1.923292e-05,-1.534938e-05,-1.006709e-05,333.566162,350.008545,348.491028,...,-0.681337,-0.776030,-0.792815,-0.658465,-0.557704,-0.456800,-0.363675,-0.245450,-0.087734,-0.006553
5,15.033587,0.000073,0.000052,1.498547e-05,1.701436e-05,-1.805706e-05,-2.294027e-06,332.904816,350.923584,348.652283,...,-0.674890,-0.759664,-0.721894,-0.608415,-0.610907,-0.541084,-0.480391,-0.333838,-0.140286,-0.038712
6,16.546116,0.000085,0.000041,1.024381e-05,6.281029e-06,-1.362825e-05,9.389391e-07,333.497345,351.693665,349.031281,...,-0.679734,-0.725434,-0.640145,-0.577411,-0.613508,-0.614491,-0.489959,-0.304273,-0.134161,-0.050201
7,17.705784,0.000079,0.000024,-2.110542e-06,-2.017151e-06,-8.375166e-06,-2.203924e-06,333.736633,351.485840,348.835083,...,-0.505346,-0.499551,-0.416206,-0.362240,-0.387235,-0.398638,-0.273317,-0.148211,-0.059976,-0.010855
8,18.141623,0.000060,0.000019,7.846455e-06,-4.622073e-06,1.237880e-06,-2.263045e-06,333.504028,350.797577,348.617981,...,-0.393108,-0.397981,-0.401111,-0.396688,-0.364644,-0.348044,-0.284693,-0.169980,-0.055227,0.012829
9,17.423697,0.000055,0.000015,1.350930e-05,2.925502e-06,1.122631e-05,-5.688536e-06,333.868713,350.352722,348.283447,...,-0.310298,-0.347083,-0.423528,-0.440026,-0.427291,-0.400578,-0.286301,-0.178496,-0.069359,-0.006471


In [116]:
ts_phailin.to_csv(output+'inner/inner_nio_all_phailin.csv')

In [117]:
maindf_lehar=pd.read_csv(target+'2013_w10max_nio_lehar.csv',delimiter=r",")
maindf_lehar.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_lehar=maindf_lehar.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2013_nio_lehar.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_lehar=pd.DataFrame.from_dict(TCrl_ts[2])
prlev_lehar=pd.DataFrame.from_dict(TCpr_ts[2])
slev_lehar=pd.DataFrame.from_dict(TCsl_ts[2])
prlev_lehar['mslp']=prlev_lehar['mslp']/100

ts1_lehar=pd.concat([maindf_lehar,extra,slev_lehar,rlev_lehar,prlev_lehar], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[2]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[2]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[2]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[2]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_lehar=pd.DataFrame.from_dict(alldict)

ts_lehar=pd.concat([ts1_lehar,plev_lehar], axis=1, join='inner')

ts_lehar

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,12.089149,-1.114373e-06,0.000028,4.712652e-05,2.683366e-05,5.358663e-06,-3.709186e-06,321.301117,347.093323,346.708557,...,-0.273026,-0.464953,-0.616295,-0.605925,-0.532822,-0.431771,-0.336090,-0.201484,-0.088615,-0.040210
1,12.099492,2.207192e-05,0.000028,4.880802e-05,3.506813e-05,3.678299e-06,-7.396683e-06,328.678925,347.569061,346.763489,...,-0.386006,-0.602521,-0.813482,-0.748966,-0.643535,-0.557331,-0.394460,-0.209155,-0.075877,-0.022236
2,12.946914,3.799148e-05,0.000048,5.157499e-05,4.190413e-05,-3.900933e-06,-1.667449e-07,330.235901,349.061005,348.044464,...,-0.590611,-0.833721,-1.013472,-0.962221,-0.953534,-0.889587,-0.608228,-0.325330,-0.105923,-0.010346
3,13.524454,6.559324e-05,0.000044,4.015736e-05,2.956255e-05,-7.985167e-06,9.848300e-07,330.619781,349.082031,348.109070,...,-0.653752,-0.828045,-0.914705,-0.866561,-0.847108,-0.756564,-0.531763,-0.303857,-0.117449,-0.039041
4,11.577554,6.630583e-05,0.000039,3.377555e-05,1.697406e-05,-1.857898e-07,-9.271409e-07,329.841675,349.316895,348.046051,...,-0.623982,-0.757471,-0.793303,-0.791738,-0.723129,-0.609871,-0.427590,-0.277627,-0.137423,-0.076792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,15.110038,2.514235e-05,-0.000003,-1.469511e-05,-3.176734e-06,1.943779e-06,8.115904e-09,328.671539,348.846008,347.335632,...,-0.051703,0.000702,-0.029430,-0.029298,-0.060750,-0.034442,-0.328356,-0.241350,-0.110749,-0.035582
65,13.975189,1.180500e-05,-0.000001,-8.951417e-06,1.838834e-06,6.610712e-06,1.255684e-06,327.732666,348.234741,346.809875,...,-0.021122,0.002817,-0.073432,-0.086392,-0.091533,-0.056784,-0.242415,-0.202245,-0.093762,-0.026863
66,13.896652,-1.453482e-07,0.000016,9.460861e-06,3.836846e-06,5.894540e-07,-8.441037e-06,322.099609,348.801941,347.279541,...,-0.085448,-0.117541,-0.167706,-0.089184,0.003859,-0.009319,-0.160035,-0.148803,-0.054789,0.002347
67,13.558363,4.308552e-06,0.000013,6.167821e-06,4.318058e-06,4.497268e-06,-5.560843e-06,322.856384,348.744019,346.840210,...,-0.059113,-0.082554,-0.141302,-0.118642,-0.059568,-0.071724,-0.180965,-0.182179,-0.073340,-0.001041


In [118]:
ts_lehar.to_csv(output+'inner/inner_nio_all_lehar.csv')

In [119]:
maindf_madi=pd.read_csv(target+'2013_w10max_nio_madi.csv',delimiter=r",")
maindf_madi.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_madi=maindf_madi.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2013_nio_madi.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_madi=pd.DataFrame.from_dict(TCrl_ts[3])
prlev_madi=pd.DataFrame.from_dict(TCpr_ts[3])
slev_madi=pd.DataFrame.from_dict(TCsl_ts[3])
prlev_madi['mslp']=prlev_madi['mslp']/100

ts1_madi=pd.concat([maindf_madi,extra,slev_madi,rlev_madi,prlev_madi], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[3]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[3]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[3]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[3]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_madi=pd.DataFrame.from_dict(alldict)

ts_madi=pd.concat([ts1_madi,plev_madi], axis=1, join='inner')

ts_madi

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,12.804338,5.682303e-05,0.000029,0.000005,2.855302e-06,9.056144e-06,0.000005,334.250183,350.075317,349.005280,...,-0.424841,-0.446905,-0.527802,-0.581141,-0.616153,-0.596044,-0.398212,-0.210010,-0.075537,-0.011303
1,14.013905,4.920553e-05,0.000027,0.000015,1.143469e-05,9.177023e-06,0.000004,335.062286,350.029358,349.248016,...,-0.442958,-0.515341,-0.576978,-0.639530,-0.697111,-0.683283,-0.446970,-0.233479,-0.082745,-0.010090
2,14.716041,5.068671e-05,0.000043,0.000033,2.169186e-05,9.461563e-06,0.000009,334.558441,351.447937,350.687195,...,-0.675027,-0.812216,-0.934410,-1.034839,-1.066676,-0.929909,-0.554989,-0.323454,-0.147879,-0.059075
3,16.741007,5.496447e-05,0.000032,0.000020,1.091589e-05,-1.053294e-06,-0.000002,335.559875,351.955933,350.899597,...,-0.685255,-0.767158,-0.779508,-0.776296,-0.684522,-0.524676,-0.354944,-0.227535,-0.089804,-0.015991
4,15.737922,4.695368e-05,0.000031,0.000010,-3.135810e-07,-3.554412e-06,-0.000007,334.122925,351.462250,350.527100,...,-0.561813,-0.573777,-0.552431,-0.495814,-0.396202,-0.336957,-0.354805,-0.241621,-0.098083,-0.021356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,15.974120,7.700743e-06,0.000008,0.000005,-6.896110e-06,-3.916429e-06,0.000002,324.595642,349.427551,346.708618,...,-0.059190,-0.047040,0.023146,0.026597,-0.033741,-0.190568,-0.309567,-0.288836,-0.122805,-0.010098
57,14.691542,-5.833215e-06,0.000006,0.000011,-8.460438e-06,-2.476413e-07,-0.000002,319.606506,349.575317,346.734558,...,0.018334,0.016591,0.059091,0.058754,0.016020,-0.112020,-0.328854,-0.307940,-0.118408,-0.008276
58,13.998476,-6.777248e-06,0.000010,0.000004,-4.489216e-06,-1.563069e-07,0.000003,314.339722,349.760559,346.952209,...,0.006835,0.013308,0.025101,0.003282,-0.052190,-0.139424,-0.342293,-0.296531,-0.122053,-0.032833
59,13.904409,3.494342e-07,0.000006,0.000003,-3.449458e-07,-5.560501e-06,0.000006,317.527100,349.655884,346.889679,...,0.009762,0.005200,0.020333,0.017817,-0.062897,-0.129848,-0.203572,-0.148956,-0.053911,-0.004697


In [120]:
ts_madi.to_csv(output+'inner/inner_nio_all_madi.csv')